## 라이브러리 및 데이터 호출

In [ ]:
# 라이브러리 호출 
import pandas as pd 
import numpy as np

In [ ]:
# 데이터 호출
data_store = pd.read_csv()
data_place = pd.read_csv()

## 1. 포화도 라벨링 기준 정하기
**경쟁강도 :: 상권 점포수 / 행정동 점포수**  
매출경쟁도 :: 상권 당월 매출 / 해당 행정동 당월 매출  
상권밀집도 :: 상권 점포수 / 상권 면적

포화지수 = (경쟁강도 * 0.4) + (매출경쟁도 * 0.4) + (상권밀집도 * 0.2)

In [ ]:
data_label = pd.merge(data_store, data_place)